<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/proyecto_apoyo/Keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MODELO PARA EXTRAER KEYWORDS

In [60]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

def cargar_modelo(model_name=MODEL_NAME):
    """
    Carga el modelo TinyLlama.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

def extraer_keywords(texto, tokenizer, model):
    """
    Dado un texto, construye un 'prompt' que pide al modelo extraer palabras clave.
    """
    prompt = ("Eres un extractor de palabras clave. Tu tarea es la siguiente:\n\n"
        "1. Lee el texto a continuación.\n"
        "2. Devuelve únicamente las TRES palabras clave más importantes.\n"
        "3. No incluyas explicaciones ni texto adicional.\n"
        "4. Sepáralas por comas y nada más.\n\n"
        f"Texto:\n{texto}\n\n"
        "Palabras clave:"
    )

    # Tokenizamos el prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    model.eval()
    # Generamos la respuesta
    with torch.no_grad():
            outputs = model.generate(
            **inputs,
            max_new_tokens=32,
            temperature=0.1,   # Al ser una tarea "extractiva", conviene un valor bajo
            do_sample=False,   # Modo determinista
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decodificamos el texto generado
    respuesta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraemos lo que viene tras "Palabras clave:"
    # Nota: Esto asume que el modelo va a contestar siguiendo el prompt.
    partes = respuesta_completa.split("Palabras clave:")
    if len(partes) > 1:
        # Tomamos la última parte (el contenido tras "Palabras clave:")
        keywords = partes[-1].strip()
    else:
        # Si no encuentra el marcador, devolvemos todo el texto
        keywords = respuesta_completa

    return keywords

if __name__ == "__main__":
    # Cargamos el modelo y el tokenizer
    tokenizer, model = cargar_modelo()


#FUNCIÓN PARA POST-PROCESAR LA SALIDA DEL MODELO

In [56]:
import re

def postprocesar_keywords(raw_keywords: str) -> str:
    """
    Toma la cadena de salida del modelo y devuelve hasta tres palabras
    clave, separadas por comas, eliminando duplicados y texto sobrante.
    """
    # 1. Dividimos por comas o saltos de línea
    #    (si el modelo genera varias palabras en línea separadas por comas o saltos).
    tokens = re.split(r"[,\n]+", raw_keywords)

    # 2. Limpiamos espacios y descartamos vacíos.
    tokens = [t.strip() for t in tokens if t.strip()]

    # 3. (Opcional) Eliminar duplicados manteniendo el orden.
    sin_duplicados = []
    seen = set()
    for token in tokens:
        if token.lower() not in seen:
            sin_duplicados.append(token)
            seen.add(token.lower())

    # 4. Quedarnos solo con los primeros 3.
    top3 = sin_duplicados[:3]

    # 5. Volver a unir en una cadena, separada por comas.
    return ", ".join(top3)


#EJEMPLOS

In [64]:
    # Texto de ejemplo
    texto_entrada = (
        "¿Qué sabes con respecto a las normativas relacionadas con la discapacidad?"
    )

    # Extraemos las palabras clave
    resultado = extraer_keywords(texto_entrada, tokenizer, model)
    # Post-procesamos para asegurar un formato limpio
    resultado = postprocesar_keywords(resultado)
    print("Palabras clave extraídas:", resultado)

Palabras clave extraídas: 1. Discapacidad, 2. Normativas, 3. Discapacidad


In [68]:
    # Texto de ejemplo
    texto_entrada = (
        "¿Dónde puedo encontrar subvenciones agrarias para jóvenes agricultores?"
    )
    # Extraemos las palabras clave
    resultado = extraer_keywords(texto_entrada, tokenizer, model)
    # Post-procesamos para asegurar un formato limpio
    resultado = postprocesar_keywords(resultado)
    print("Palabras clave extraídas:", resultado)

Palabras clave extraídas: agricultura, jóvenes, subvenciones


In [67]:
    # Texto de ejemplo
    texto_entrada = (
        "Necesito información sobre la ley de protección de datos en empresas tecnológicas."
    )
    # Extraemos las palabras clave
    resultado = extraer_keywords(texto_entrada, tokenizer, model)
    # Post-procesamos para asegurar un formato limpio
    resultado = postprocesar_keywords(resultado)
    print("Palabras clave extraídas:", resultado)

Palabras clave extraídas: protección, datos, tecnología


In [66]:
    # Texto de ejemplo
    texto_entrada = (
        "Requiero documentación acerca de la normativa ambiental que prohíba vertidos al río."
    )
    # Extraemos las palabras clave
    resultado = extraer_keywords(texto_entrada, tokenizer, model)
    # Post-procesamos para asegurar un formato limpio
    resultado = postprocesar_keywords(resultado)
    print("Palabras clave extraídas:", resultado)

Palabras clave extraídas: Normativa, ambiental, vertidos
